In [40]:
import pandas as pd
import numpy as np
from tools import acc_score

In [2]:
df_train = pd.read_csv("../data/train.csv", index_col=0)
df_test = pd.read_csv("../data/test.csv", index_col=0)

In [244]:
train_bins = seq_to_num(df_train.Sequence, target_split=True, pad=True, pad_adaptive=True,
                        pad_maxlen=100, dtype=np.float32, drop_na_inf=True,
                        nbins=5, bins_by='terms')
test_bins = seq_to_num(df_test.Sequence, target_split=True, pad_adaptive=True,
                            dtype=np.float32, drop_na_inf=True, nbins=5, bins_by='terms')

In [223]:
train_X, train_y, _ = train_bins[4]

In [224]:
test_X, test_y, test_idx = test_bins[4]

In [225]:
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor

In [226]:
dt = DecisionTreeRegressor(random_state=42)
dt.fit(train_X, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [228]:
acc_score(dt.predict(test_X), test_y)

0.25655293600820034

In [140]:
train_X2, train_y2, _ = train_bins[1]
test_X2, test_y2, _ = test_bins[1]

In [141]:
etr = ExtraTreeRegressor(max_depth=100, random_state=42)
etr.fit(train_X2, train_y2)

ExtraTreeRegressor(criterion='mse', max_depth=100, max_features='auto',
          max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=42, splitter='random')

In [142]:
acc_score(etr.predict(test_X2), test_y2)

0.09251939686722295

In [158]:
# too long sequence?
train_X2, train_y2 = train_bins[2]
test_X2, test_y2 = test_bins[2]
etr = DecisionTreeRegressor(max_depth=5, random_state=42)
etr.fit(train_X2, train_y2)
acc_score(etr.predict(test_X2), test_y2)

0.0

In [159]:
from sklearn.neural_network import MLPRegressor

In [160]:
# NNet still doesn't work
mlp = MLPRegressor(hidden_layer_sizes=(10, 1))
mlp.fit(train_X, train_y)
acc_score(mlp.predict(test_X), test_y)

/Users/ptyshevs/envs/loc_env/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.0

In [162]:
# Try to combine predictions for bin 3 and 4 (by terms), while
# fallback to mode on bin 0, 1, 2
def mmode(arr):
    modes = []
    for row in arr:
        counts = {i: row.tolist().count(i) for i in row}
        if len(counts) > 0:
            modes.append(max(counts.items(), key=lambda x:x[1])[0])
        else:
            modes.append(0)
    return modes

In [43]:
kg_train = pd.read_csv('../data/kaggle_train.csv', index_col=0)
kg_test = pd.read_csv('../data/kaggle_test.csv', index_col=0)

In [229]:
train_bins = seq_to_num(kg_train.Sequence, target_split=True,
                        pad_adaptive=True, dtype=np.float32, drop_na_inf=True,
                        nbins=5, bins_by='terms')
test_bins = seq_to_num(kg_test.Sequence, target_split=False, pad_adaptive=True,
                            dtype=np.float32, drop_na_inf=True, nbins=5, bins_by='terms')

In [230]:
bin3_X, bin3_y, _ = train_bins[3]
bin4_X, bin4_y, _ = train_bins[4]
dt_bin3 = DecisionTreeRegressor(random_state=42)
dt_bin4 = DecisionTreeRegressor(random_state=42)
dt_bin3.fit(bin3_X, bin3_y)
dt_bin4.fit(bin4_X, bin4_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [232]:
pred_bin3 = dt_bin3.predict(test_bins[3][0])
pred_bin4 = dt_bin4.predict(test_bins[4][0])

In [237]:
test_bins[3][1].shape, pred_bin3.shape

((22764,), (22764,))

In [173]:
pred_bin0 = mmode(test_bins[0])

In [174]:
pred_bin1 = mmode(test_bins[1])
pred_bin2 = mmode(test_bins[2])

In [238]:
pred3 = pd.Series(pred_bin3, index=test_bins[3][1], dtype=object).map(lambda x: int(x))
pred4 = pd.Series(pred_bin4, index=test_bins[4][1], dtype=object).map(lambda x: int(x))

In [240]:
pred_total = pd.Series(np.zeros(kg_test.shape[0]), index=kg_test.index, dtype=np.int64)
pred_total[test_bins[3][1]] = pred_bin3
pred_total[test_bins[4][1]] = pred_bin4

In [241]:
prep_submit(pred_total)